Clase ComplejoSimplicial con sus métodos principales

In [302]:
from itertools import chain, combinations

class ComplejoSimplicial:

    def __init__(self, sim_maximales):
        """
        Inicializa el complejo simplicial tomando como argumento lo que serían los maximales del poset
        """
        self.simplices = dict()
        self.simplices_ordenados = [] # Nuevo atributo para guardar los simplices ordenados por flotante y dimensión
        sim_derivados = self.derivar_simplices(sim_maximales)
        for sim in sim_derivados:
            if isinstance(sim, int):
                dim = 0
            else:    
                dim = len(sim)-1
            if dim not in self.simplices:
                self.simplices[dim] = set()
            self.simplices[dim].add(sim)

    def derivar_simplices(self, simplices_maximales):
        """
        Genera el complejo simplicial tomando todos los subconjuntos de los símplices maximales.
        
        Args:
        simplices_maximales (list of tuples): Lista de símplices maximales.
        
        Returns:
        set: El conjunto de todos los símplices en el complejo simplicial.
        """
        complejo = set()
        for simplice in simplices_maximales:
            complejo.update(self.potencia(simplice))
        return complejo
    
    def potencia(self, simplice):
        """
        Genera todos los subconjuntos(caras) de un simplice (incluido el propio simplice).
        
        Args:
        simplice (tuple): Un simplice representado como una tupla de vértices.
        
        Returns:
        set: Conjunto de todos los subconjuntos del simplice.
        """
        return set(chain.from_iterable(combinations(simplice, r) for r in range(len(simplice) + 1)))
    
    def insert(self, simplices, value):
        """
        Añade nuevos símplices al complejo simplicial con un valor flotante asociado.
        
        Args:
        simplices (list of tuples): Lista de símplices representados como tuplas de vértices.
        value (float): Valor flotante a asociar con cada simplice.
        """
        for simplice in simplices:
            nuevos_simplices = self.potencia(simplice)
            for sim in nuevos_simplices:
                if isinstance(sim, int):
                    dim = 0
                else:
                    dim = len(sim) - 1
                if dim not in self.simplices:
                    self.simplices[dim] = set()
                self.simplices[dim].add((sim, value))
                # Añadir a la lista de simplices ordenados
                self.simplices_ordenados.append((sim, dim, value))

        # Ordenar los símplices primero por valor flotante, luego por dimensión
        self.simplices_ordenados.sort(key=lambda x: (x[2], x[1]))

    def filtracion(self, umbral):
        """
        Devuelve un nuevo complejo simplicial formado por todos los símplices cuyo valor flotante asociado 
        sea menor o igual que el flotante dado.
        
        Args:
        umbral (float): El valor máximo del flotante permitido en la filtración.
        
        Returns:
        ComplejoSimplicial: Un nuevo complejo simplicial con simplices filtrados.
        """
        simplices_filtrados = []
        for simplice, dim, value in self.simplices_ordenados:
            if value <= umbral:
                simplices_filtrados.append(simplice)
        
        # Crear un nuevo complejo simplicial con los símplices filtrados
        return ComplejoSimplicial(simplices_filtrados)
  
    def dim(self):
        """
        Devuelve la dimension del complejo simplicial, que es el maximo de las dimensiones de los simplices.
        """
        return max(self.simplices.keys())

    
    def caras(self, dim):
        """
        Devuelve una lista con las caras de un complejo simplicial para una dimension dada
        """
        caras = []
        for cara in self.simplices[dim]:
            caras.append(cara)
        return caras

    
    def face_set(self):
        """
        Devuelve todas las caras del complejo simplicial, ignorando los flotantes asociados.
        """
        def extract_simplice(simplice):
            # Si el simplice es una tupla no vacía con un flotante, extrae solo la parte entera
            if isinstance(simplice, tuple) and len(simplice) > 0 and isinstance(simplice[-1], float):
                return simplice[:-1]
            return simplice
    
        caras = set()
        for dim in self.simplices:
            for cara in self.simplices[dim]:
                caras.add(extract_simplice(cara))

        # Eliminar el conjunto vacío
        return set(cara for cara in caras if cara != ())

    def estrella(self, simplice):
        """
        Devuelve la estrella de un simplice dado
        Se buscan las caras que contienen completamente a dicho simplice
        """
        simplice = set(simplice)  # Convertimos el simplice en un conjunto(set), para poder hacer intersecciones
        # Caras cuya intersección con el simplice es el simplice
        return set(cara for cara in self.face_set() if simplice.intersection(cara) == simplice)
    
    
    def estrella_cerrada(self, simplex):
        """
        Devuelve la estrella cerrada de un simplice dado.
        La estrella cerrada es el menor subcomplejo simplicial que contiene a la estrella de un simplice dado.
        """
        # Obtenemos la estrella del simplex dado
        estrella = self.estrella(simplex)
        # Inicializamos la estrella cerrada como un conjunto vacío
        estrella_cerrada = set()
        cubiertos = set()  # Para rastrear qué simplices de la estrella están cubiertos
        
        # Recorremos todos los símplices de menor a mayor dimensión
        for dim in sorted(self.simplices.keys()):
            for cara in self.simplices[dim]:
                cara_set = set(cara)
                
                # Verificamos si la cara cubre algunos simplices de la estrella
                if any(set(s).issubset(cara_set) for s in estrella):
                    estrella_cerrada.add(cara)
                    # Actualizamos los simplices de la estrella que están cubiertos
                    cubiertos.update(s for s in estrella if set(s).issubset(cara_set))
                
                # Si ya hemos cubierto todos los simplices de la estrella, podemos detenernos
                if cubiertos == estrella:
                    break
        
        return estrella_cerrada
  
    
    def link(self, simplice):
        """
        Devuelve el link del simplice dado
        El link de un simplice es el conjunto de simplices de la estrella cerrada con interseccion vacia con el simplice.
        """
        # Obtenemos la estrella cerrada del simplice.
        estrella_cer = self.estrella_cerrada(simplice)
        # Convertimos en complejo simplicial, para poder obtener todas sus  con face_set()
        ec = ComplejoSimplicial(estrella_cer)
        # Nos quedamos con los simplices de la estrella cerrada que no intersecan con el simplice del que queremos obtener el link
        return set(cara for cara in ec.face_set() if not set(simplice).intersection(cara))

    
    def print_caras(self):
        """
        Permite printear el complejo simplicial por dimensiones
        """
        dim = self.dim()
        for i in range(0,dim+1):
            print("Caras de dim: ", i)
            d_caras = self.caras(i)
            for cara in d_caras:
                print(cara)
    
    
    def carac_euler(self):
        """
        Calcula la característica de Euler del complejo simplicial.
        Consiste en la suma alternante de los números de simplices de cada dimensión.
        """
        """ dim = self.dim()
        res = 0
        for i in range(0, dim+1):
            if i % 2 == 0:
                res += len(self.simplices[dim])
            else:
                res -= len(self.simplices[dim])
        return res """
        # Forma más eficiente
        return sum((-1)**k * len(self.simplices[k]) for k in range(self.dim()+1))
    

    def num_componentes_conexas(self):
        """
        Devuelve el número de componentes conexas del complejo simplicial
        """
        n = 0
        vertices = list(self.simplices[0])
        aristas = self.simplices[1]

        v_conexos = []
        v_inconexos = []
        v0 = vertices[0]

        while True:
            #print('hola')
            for v in vertices[:]:
                print(f"v0: {v0}, v: {v}, existe_camino: {self.existe_camino(v0[0], v[0], aristas)}")
                if self.existe_camino(v0[0], v[0], aristas):
                    v_conexos.append(v)
                    vertices.remove(v)
                else:
                    v_inconexos.append(v)
            
            n += 1
            if len(vertices) == 0:
                return n
            v0 = vertices[0]
            v_conexos = []
            v_inconexos = []


    def existe_camino(self, v, u, aristas):
        #print(aristas)
        # Crear el grafo como un diccionario de adyacencia
        grafo = dict()
        
        # Construir el diccionario de adyacencia a partir de las aristas
        for v1, v2 in aristas:
            if v1 not in grafo.keys():
                grafo[v1] = []
            if v2 not in grafo:
                grafo[v2] = []
            grafo[v1].append(v2)
            grafo[v2].append(v1)  # Como es un grafo no dirigido, añadir ambas conexiones

        # Función recursiva para la búsqueda en profundidad (DFS)
        def bep(vertice, visitados):
            if vertice == u:
                return True
            visitados.append(vertice)  # Marcar el vértice como visitado
            # Recorrer los vecinos
            if vertice in grafo.keys():
                for vecino in grafo[vertice]:  # Obtener vecinos del vértice
                    if vecino not in visitados:  # Solo visitar los no visitados
                        if bep(vecino, visitados):
                            return True
            
            return False
        
        return bep(v, list())
    
    def skeleton(self, k):
        """
        Devuelve el esqueleto de dimensión k del complejo simplicial.
        
        Args:
        k (int): La dimensión máxima de los simplices en el esqueleto.
        
        Returns:
        ComplejoSimplicial: Un nuevo complejo simplicial que es el esqueleto de dimensión k.
        """
        # Recopilar todas las caras de dimensión k o menor
        caras_k_o_menor = set()
        for dim in range(k + 1):
            if dim in self.simplices:
                caras_k_o_menor.update(self.simplices[dim])
        
        # Crear un nuevo complejo simplicial con estas caras
        return ComplejoSimplicial(list(caras_k_o_menor))
    
    def treshold(self, simplice):
        """
        Devuelve el valor mínimo asociado al simplice dado (donde el simplice aparece por primera vez).
        
        Args:
        simplice (tuple): Un simplice representado como una tupla de vértices.
        
        Returns:
        float: El valor flotante mínimo asociado al simplice, o None si no se encuentra.
        """
        min_value = None
        
        for dim in self.simplices:
            for cara in self.simplices[dim]:
                if isinstance(cara, tuple) and set(simplice).issubset(cara[0]):
                    value = cara[1]
                    # Guardamos el valor mínimo
                    if min_value is None or value < min_value:
                        min_value = value
                        
        return min_value



EJEMPLO 1: Tetraedro

In [303]:
# Definimos un complejo simplicial
sc = ComplejoSimplicial([(0,1,2,3)])

In [304]:
# Conjunto de todas las caras del complejo simplicial
sc.print_caras()

Caras de dim:  0
(0,)
(1,)
(2,)
(3,)
Caras de dim:  1
(0, 1)
(1, 2)
(0, 3)
(2, 3)
(0, 2)
(1, 3)
Caras de dim:  2
(0, 1, 2)
(0, 1, 3)
(1, 2, 3)
(0, 2, 3)
Caras de dim:  3
(0, 1, 2, 3)


In [305]:
# Dimensión del complejo simplicial
sc.dim()

3

In [306]:
# Vértices del complejo simplicial
sc.caras(0)

[(0,), (1,), (2,), (3,)]

In [307]:
# Aristas del complejo simplicial
sc.caras(1)

[(0, 1), (1, 2), (0, 3), (2, 3), (0, 2), (1, 3)]

In [308]:
# 2-símplices del complejo simplicial
sc.caras(2)

[(0, 1, 2), (0, 1, 3), (1, 2, 3), (0, 2, 3)]

In [309]:
# 3-símplices del complejo simplicial
sc.caras(3)

[(0, 1, 2, 3)]

In [310]:
# Característica de Euler (4-6+4=2)
sc.carac_euler()

1

In [311]:
# Estrella de la arista (0,1)
sc.estrella([0,1])

{(0, 1), (0, 1, 2), (0, 1, 2, 3), (0, 1, 3)}

In [312]:
# Estrella cerrada de la arista (0,1)
# Como el total está en la estrella, la estrella cerrada claramente es el total
sc.estrella_cerrada([0,1])

{(0, 1), (0, 1, 2), (0, 1, 2, 3), (0, 1, 3)}

In [313]:
# Link de la arista (0,1)
sc.link([0,1])

{(2,), (2, 3), (3,)}

In [314]:
# Componentes conexas
sc.num_componentes_conexas()

v0: (0,), v: (0,), existe_camino: True
v0: (0,), v: (1,), existe_camino: True
v0: (0,), v: (2,), existe_camino: True
v0: (0,), v: (3,), existe_camino: True


1

EJEMPLO 2: Borde del tetraedro

In [253]:
sc1=sc.skeleton(2)

In [123]:
# Conjunto de todas las caras del complejo simplicial
sc1.print_caras()

Caras de dim:  0
(0,)
(1,)
(2,)
(3,)
Caras de dim:  1
(0, 1)
(1, 2)
(0, 3)
(2, 3)
(0, 2)
(1, 3)
Caras de dim:  2
(0, 1, 2)
(0, 1, 3)
(1, 2, 3)
(0, 2, 3)


In [124]:
sc1.dim()

2

In [254]:
# Estrella del vértice 0
sc1.estrella((0,))

{(0,), (0, 1), (0, 1, 2), (0, 1, 3), (0, 2), (0, 2, 3), (0, 3)}

In [255]:
sc1.estrella_cerrada((0,))

{(0,), (0, 1), (0, 1, 2), (0, 1, 3), (0, 2), (0, 2, 3), (0, 3)}

In [256]:
# Link del vértice 0
sc1.link((0,))

{(1,), (1, 2), (1, 3), (2,), (2, 3), (3,)}

In [128]:
# Característica de Euler 
sc1.carac_euler()

2

In [91]:
# Componentes conexas
sc1.num_componentes_conexas()

v0: (0,), v: (0,), existe_camino: True
v0: (0,), v: (1,), existe_camino: True
v0: (0,), v: (2,), existe_camino: True
v0: (0,), v: (3,), existe_camino: True


1

EJEMPLO 3

In [151]:
sc2 =ComplejoSimplicial([(0,1),(1,2,3,4),(4,5),(5,6),(4,6),(6,7,8),(8,9)])

In [130]:
# Conjunto de todas las caras del complejo simplicial
sc2.print_caras()

Caras de dim:  0
(6,)
(2,)
(5,)
(8,)
(4,)
(1,)
(7,)
(0,)
(3,)
(9,)
Caras de dim:  1
(0, 1)
(2, 4)
(1, 2)
(3, 4)
(6, 8)
(4, 6)
(1, 4)
(2, 3)
(6, 7)
(4, 5)
(8, 9)
(5, 6)
(1, 3)
(7, 8)
Caras de dim:  2
(1, 2, 3)
(1, 3, 4)
(6, 7, 8)
(2, 3, 4)
(1, 2, 4)
Caras de dim:  3
(1, 2, 3, 4)


In [152]:
sc2.dim()

3

In [153]:
# 1-esqueleto
sc2.skeleton(1).print_caras()

Caras de dim:  0
(6,)
(2,)
(5,)
(8,)
(4,)
(1,)
(7,)
(0,)
(3,)
(9,)
Caras de dim:  1
(0, 1)
(2, 4)
(1, 2)
(3, 4)
(6, 8)
(4, 6)
(1, 4)
(2, 3)
(6, 7)
(4, 5)
(8, 9)
(5, 6)
(1, 3)
(7, 8)


In [154]:
# Estrella del vértice 4
sc2.estrella((4,))

{(1, 2, 3, 4),
 (1, 2, 4),
 (1, 3, 4),
 (1, 4),
 (2, 3, 4),
 (2, 4),
 (3, 4),
 (4,),
 (4, 5),
 (4, 6)}

In [155]:
# Link del vértice 4
sc2.link((4,))

{(1,), (1, 2), (1, 2, 3), (1, 3), (2,), (2, 3), (3,), (5,), (6,)}

In [156]:
# Característica de Euler
sc2.carac_euler()

0

In [99]:
# Componentes conexas
sc2.num_componentes_conexas()

v0: (6,), v: (6,), existe_camino: True
v0: (6,), v: (2,), existe_camino: True
v0: (6,), v: (5,), existe_camino: True
v0: (6,), v: (8,), existe_camino: True
v0: (6,), v: (4,), existe_camino: True
v0: (6,), v: (1,), existe_camino: True
v0: (6,), v: (7,), existe_camino: True
v0: (6,), v: (0,), existe_camino: True
v0: (6,), v: (3,), existe_camino: True
v0: (6,), v: (9,), existe_camino: True


1

EJEMPLO 4

In [100]:
sc3 = sc2.skeleton(1)

In [101]:
sc3.carac_euler()

-4

EJEMPLO 5

EJEMPLO 6

In [315]:
sc6 = ComplejoSimplicial([(1,2,4),(1,3,6),(1,4,6),(2,3,5),(2,4,5),(3,5,6)])

In [316]:
sc6.face_set()

{(1,),
 (1, 2),
 (1, 2, 4),
 (1, 3),
 (1, 3, 6),
 (1, 4),
 (1, 4, 6),
 (1, 6),
 (2,),
 (2, 3),
 (2, 3, 5),
 (2, 4),
 (2, 4, 5),
 (2, 5),
 (3,),
 (3, 5),
 (3, 5, 6),
 (3, 6),
 (4,),
 (4, 5),
 (4, 6),
 (5,),
 (5, 6),
 (6,)}

In [317]:
sc6.dim()

2

In [318]:
sc6.estrella((1,4))

{(1, 2, 4), (1, 4), (1, 4, 6)}

In [319]:
sc6.link((1,4))

{(2,), (6,)}

In [320]:
sc6.carac_euler()

0

Hay hasta ejemplo 11...

Filtración de complejos simpliciales

In [321]:
sc = ComplejoSimplicial([])
sc.insert([(0,1)], 1.0)
sc.insert([(1,2), (2,3), (2,4)], 2.0)
sc.insert([(3,4)], 3.0)
sc.insert([(2,3,4)], 4.0)

# TODO: No consigo que no sagan los ")" que sobran
sc.face_set()

{((),),
 ((0,),),
 ((0, 1),),
 ((1,),),
 ((1, 2),),
 ((2,),),
 ((2, 3),),
 ((2, 3, 4),),
 ((2, 4),),
 ((3,),),
 ((3, 4),),
 ((4,),)}

In [322]:
# El siguiente comando devuelve el parámetro (umbral) para el que aparece un símplice. Vemos cuando aparece el vétice 3.
sc.treshold((3,))

2.0

In [323]:
K1=sc.filtracion(1.0)
K2=sc.filtracion(2.0)
K3=sc.filtracion(3.0)
K4=sc.filtracion(4.0)

K1.face_set()

{(0,), (0, 1), (1,)}

In [324]:
K2.face_set()

{(0,), (0, 1), (1,), (1, 2), (2,), (2, 3), (2, 4), (3,), (4,)}

In [325]:
K3.face_set()

{(0,), (0, 1), (1,), (1, 2), (2,), (2, 3), (2, 4), (3,), (3, 4), (4,)}

In [326]:
K4.face_set()

{(0,),
 (0, 1),
 (1,),
 (1, 2),
 (2,),
 (2, 3),
 (2, 3, 4),
 (2, 4),
 (3,),
 (3, 4),
 (4,)}